In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [22]:
trn_id=pd.read_csv('train_identity.csv') 
trn_trans=pd.read_csv('train_transaction.csv')
trn_data=pd.merge(trn_id, trn_trans, on='TransactionID', how='outer')

In [23]:
import gc
del trn_id,trn_trans
gc.collect()

46

In [24]:
x=trn_data[trn_data["isFraud"].notna()]

In [25]:
 x= x.loc[:, x.isnull().sum() < 0.75*x.shape[0]]

In [26]:
del trn_data
gc.collect()

22

In [27]:
y=x.pop("isFraud")

In [28]:
ff=[c for c in x.columns if  x[c].isnull().sum() < 0.80 * x.shape[0]]

In [29]:
cat_col= [c for c in ff if
                   x[c].dtype == "object"]


num_col = [c for c in ff if 
                  c not in cat_col]

In [30]:
tst_id=pd.read_csv('test_identity.csv') 
tst_trans=pd.read_csv('test_transaction.csv')
tst_data=pd.merge(tst_id, tst_trans, on='TransactionID', how='outer')
del tst_id,tst_trans
gc.collect()

0

In [31]:
tst_data.rename({'id-01':'id_01','id-02':'id_02','id-03':'id_03','id-04':'id_04','id-05':'id_05','id-06':'id_06','id-07':'id_07','id-08':'id_08','id-09':'id_09','id-10':'id_10','id-11':'id_11','id-12':'id_12','id-13':'id_13','id-14':'id_14','id-15':'id_15','id-16':'id_16','id-17':'id_17','id-18':'id_18','id-19':'id_19','id-20':'id_20','id-21':'id_21','id-22':'id_22','id-23':'id_23','id-24':'id_24','id-25':'id_25','id-26':'id_26','id-27':'id_27','id-28':'id_28','id-29':'id_29','id-30':'id_30','id-31':'id_31', 'id-32':'id_32', 'id-33':'id_33', 'id-34':'id_34', 'id-35':'id_35', 'id-36':'id_36', 'id-37':'id_37', 'id-38':'id_38'}, axis=1, inplace=True)

In [32]:
x=pd.concat((x[cat_col],x[num_col]),axis=1)

In [33]:
tst_out=pd.concat((tst_data[cat_col],tst_data[num_col]),axis=1)

In [34]:
transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"), # there are a few missing values
    StandardScaler(),
)
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(sparse=False,handle_unknown='ignore'),
    #OrdinalEncoder()
)

preprocessor = make_column_transformer(
    (transformer_num, num_col),
    (transformer_cat, cat_col),
)

In [35]:
x = preprocessor.fit_transform(x)
tst_out = preprocessor.transform(tst_out)

In [36]:
x.shape

(590540, 315)

In [37]:
tst_out.shape

(506691, 315)

In [38]:
df=pd.DataFrame(x)
df.to_csv('x_train.csv',index=False)
df=pd.DataFrame(y)
df.to_csv('y_train.csv',index=False)
df=pd.DataFrame(tst_out)
df.to_csv('ftst.csv',index=False)